In [69]:
import numpy as np
import pandas as pd
from scipy.linalg import eigh
from tqdm.auto import tqdm

import seaborn as sns
import matplotlib.pyplot as plt

from qiskit import transpile
from qiskit.circuit.library import UnitaryGate
from qiskit_aer import Aer

import utils
# good seeds (9112001, 9112005)
np.random.seed(9112005)
np.set_printoptions(precision=5)

In [70]:
# consider 3 qubits
N   = 3

""" Build A matrix such that the Hamiltoninan has
    entries with both real and imaginary part 
    inside (-bound, bound)
"""
bound = 1
H = utils.get_random_H(N, bound)

evs, w = eigh(H)
w      = w.T
gs     = w[0] / np.linalg.norm(w[0]) # we need to normalize it because later we calculate eps
w_max  = w[np.argmax(np.abs(evs))] # select eigenvector that corresponds to the eigenvalue of maximum module

evect_idx = 7
w1     = w[evect_idx] / np.linalg.norm(w[evect_idx])

v_min = np.min(np.abs(evs))
v_max = np.max(np.abs(evs))
print(f"Smallest eigenvalue: {evs[0]:.5f}")
print(f"Biggest eigenvalue: {evs[-1]:.5f}")
print(f"Smallest eigenvalue in magnitude v_min (index = {np.argmin(np.abs(evs))}): {v_min:.5f}")
print(f"Biggest eigenvalue in magnitude v_max (index = {np.argmax(np.abs(evs))}): {v_max:.5f}")
print(f"gs is w_max: {np.allclose(gs, w_max)}")
print(f"Eval associated to evect with index = {evect_idx} (w1): {evs[evect_idx]:.5f}")
print(f"w1 is w_max: {np.allclose(w1, w_max)}")

psi_0 = np.ones(2 ** N)
psi_0 /= np.linalg.norm(psi_0)
print(f"sp with the eigenstate associated with the largest eigenvalue: {np.abs(np.vdot(w[-1], psi_0)) ** 2}")

Smallest eigenvalue: -2.36533
Biggest eigenvalue: 3.12976
Smallest eigenvalue in magnitude v_min (index = 3): 0.44265
Biggest eigenvalue in magnitude v_max (index = 7): 3.12976
gs is w_max: False
Eval associated to evect with index = 7 (w1): 3.12976
w1 is w_max: True
sp with the eigenstate associated with the largest eigenvalue: 0.07887530053515387


In [71]:
M_max    = 4
M_values = np.arange(1, M_max + 1)
n_points = 6
a_values = np.linspace(0, 1, n_points)

data = []

for M in tqdm(M_values):
    n_anc = 2 * M
    t = M / (2 * v_max) # moving along the straight line in the heatmap

    qc = utils.get_itimevol_circuit(N, M, H, t, psi_0)
    qc.save_statevector()

    aer_sim = Aer.get_backend("aer_simulator_statevector")
    qc_tp   = transpile(qc, backend=aer_sim)
    result  = aer_sim.run(qc_tp, shots=1).result()

    final_state_dict     = result.get_statevector(qc_tp).to_dict()
    final_state_sys_dict = {}

    for k, v in final_state_dict.items():
        if k[-n_anc:] == '0' * n_anc:
            k_new = k[:N]
            final_state_sys_dict[k_new] = v

    final_state_sys_list = [(k, v) for k, v in final_state_sys_dict.items()]
    final_state_sys_list = sorted(final_state_sys_list)
    final_state          = np.array([t[1] for t in final_state_sys_list])
    final_state          /= np.linalg.norm(final_state)

    fs = np.abs(np.vdot(gs, final_state)) ** 2
    data.append((M, fs))

100%|██████████| 4/4 [00:08<00:00,  2.15s/it]


In [72]:
df = pd.DataFrame(data, columns=["M", "fs"])
print(df)

   M        fs
0  1  0.440329
1  2  0.281693
2  3  0.185942
3  4  0.129422
